In [1]:
import sys
sys.path.append('../utils')

In [2]:
from spark_config import spark_configs

spark = spark_configs()

23/04/25 20:49:42 WARN Utils: Your hostname, thiagovale-550XDA resolves to a loopback address: 127.0.1.1; using 192.168.0.15 instead (on interface wlo1)
23/04/25 20:49:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/thiago-vale/.ivy2/cache
The jars for the packages stored in: /home/thiago-vale/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-cf95040a-69ef-44ca-ab94-f5ca4e3b5780;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.0.2 in central
	found io.delta#delta-storage;2.0.2 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
:: resolution report :: resolve 128ms :: artifacts dl 5ms
	:: modules in use:
	io.delta#delta-core_2.12;2.0.2 from central in [default]
	io.delta#delta-storage;2.0.2 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	org.codehaus.jackson#jackson-core-asl;1.9.13 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|d

In [3]:
spark = spark.builder.appName('read spark').getOrCreate()

In [6]:
df = spark.read.format('csv').option('header','true').option('inferSchema','true').option('delimiter',';').load('../data/MICRODADOS_ENADE_2017.csv')

In [12]:
df.show(1)

+------+------+----------+----------+--------+--------+-------------+--------------+-----------+---------------+--------+-------+----------+-----------+------------------+----------------+------------+-----------+-------------+-------------+-------------+-----------+-------------+-------------+-------------+------------------+-----------------+--------------------+--------------------+-------------+-------------+--------------------+--------------------+-------+---------+-----------+-----------+-----------+-----------+---------+---------+---------+---------+---------+------+-----+---------+---------+--------+-----------+-----------+--------+-----------+-----------+-----+---------+---------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+----

In [13]:
df.printSchema()

root
 |-- NU_ANO: integer (nullable = true)
 |-- CO_IES: integer (nullable = true)
 |-- CO_CATEGAD: integer (nullable = true)
 |-- CO_ORGACAD: integer (nullable = true)
 |-- CO_GRUPO: integer (nullable = true)
 |-- CO_CURSO: integer (nullable = true)
 |-- CO_MODALIDADE: integer (nullable = true)
 |-- CO_MUNIC_CURSO: integer (nullable = true)
 |-- CO_UF_CURSO: integer (nullable = true)
 |-- CO_REGIAO_CURSO: integer (nullable = true)
 |-- NU_IDADE: integer (nullable = true)
 |-- TP_SEXO: string (nullable = true)
 |-- ANO_FIM_EM: integer (nullable = true)
 |-- ANO_IN_GRAD: integer (nullable = true)
 |-- CO_TURNO_GRADUACAO: integer (nullable = true)
 |-- TP_INSCRICAO_ADM: integer (nullable = true)
 |-- TP_INSCRICAO: integer (nullable = true)
 |-- NU_ITEM_OFG: integer (nullable = true)
 |-- NU_ITEM_OFG_Z: integer (nullable = true)
 |-- NU_ITEM_OFG_X: integer (nullable = true)
 |-- NU_ITEM_OFG_N: integer (nullable = true)
 |-- NU_ITEM_OCE: integer (nullable = true)
 |-- NU_ITEM_OCE_Z: intege

In [14]:
df.createTempView('temp')

In [32]:
resposta = spark.sql('''
                        select count(*)
                        from temp
                        where CO_GRUPO in ('3201','3202')
                            ''')

In [33]:
resposta.show()

+--------+
|count(1)|
+--------+
|    7113|
+--------+



In [36]:
resposta = spark.sql('''
                        select count(*)
                        from temp
                        where CO_GRUPO in ('3201','3202')
                        and TP_SEXO = 'M'
                            ''')

In [37]:
resposta.show()

+--------+
|count(1)|
+--------+
|    4533|
+--------+



In [79]:
resposta = spark.sql('''
                        select 
                        round(cast(regexp_replace(NT_GER, ',', '.')as double),2) as NT_GER
                        from temp
                        where CO_GRUPO in ('3201','3202')
                        group by 1
                        order by 2 DESC
                            ''')

In [80]:
resposta.show(40)

+-----------+------+
|CO_UF_CURSO|NT_GER|
+-----------+------+
|         53|  53.7|
|         33| 50.59|
|         42| 49.99|
|         31| 49.87|
|         52| 48.42|
|         35| 48.28|
|         26| 48.19|
|         50| 48.17|
|         41| 48.16|
|         43| 47.56|
|         24| 45.48|
|         16| 44.23|
|         32| 44.04|
|         29|  44.0|
|         25| 43.83|
|         28| 43.04|
|         15| 41.79|
|         27| 41.59|
|         51|  41.4|
|         23| 40.98|
|         17| 40.34|
|         13| 40.22|
|         14| 38.62|
|         21| 36.13|
|         11| 35.59|
|         12| 34.81|
|         22|  33.6|
+-----------+------+



In [120]:
resposta1 = spark.sql('''
    SELECT
    AVG(regexp_replace(NU_IDADE, ',', '.')) as NT_GER
    FROM temp
    WHERE CO_GRUPO IN ('3201','3202')
''')

resposta1.show()


+------------------+
|            NT_GER|
+------------------+
|33.669619007451146|
+------------------+



In [128]:
resposta2 = spark.sql('''
    SELECT
    AVG(regexp_replace(NU_IDADE, ',', '.')) as NT_GER
    FROM temp
    WHERE CO_GRUPO IN ('1401','1402')
''')

resposta2.show()

+------------------+
|            NT_GER|
+------------------+
|28.099538562953196|
+------------------+



In [109]:
resposta = spark.sql('''
                        select 
                        round(cast(regexp_replace(NT_GER, ',', '.')as double),0) as NT_GER
                        from temp
                        where CO_GRUPO in ('3201','3202')
                        and CO_RS_I2 = 'B'
                        order by 1 desc
                            ''')

In [110]:
resposta.show()

+------+
|NT_GER|
+------+
|  90.0|
|  86.0|
|  85.0|
|  81.0|
|  80.0|
|  79.0|
|  79.0|
|  78.0|
|  76.0|
|  75.0|
|  75.0|
|  75.0|
|  75.0|
|  75.0|
|  74.0|
|  74.0|
|  73.0|
|  73.0|
|  73.0|
|  72.0|
+------+
only showing top 20 rows



In [126]:
resposta1 = spark.sql('''
SELECT AVG(CAST(regexp_replace(NT_GER, ',', '.') AS DOUBLE)) as media_nt_ger
FROM temp
WHERE CO_UF_CURSO = '43' AND CO_GRUPO IN ('3201', '3202')
''')

resposta1.show()

+-----------------+
|     media_nt_ger|
+-----------------+
|47.56138888888884|
+-----------------+



In [136]:
resposta1 = spark.sql('''
    SELECT
    CO_UF_CURSO,
    AVG(NU_IDADE) 
    FROM temp
    WHERE CO_GRUPO IN ('3201','3202')
    group by 1
    order by 2 desc
''')

resposta1.show()

+-----------+------------------+
|CO_UF_CURSO|     avg(NU_IDADE)|
+-----------+------------------+
|         32|  37.9920424403183|
|         11|37.714285714285715|
|         42| 37.66773162939297|
|         21| 36.94009216589862|
|         15|35.815384615384616|
|         43|35.669623059866964|
|         29|34.836120401337794|
|         25| 33.89102564102564|
|         33|33.691466083150985|
|         53|33.614583333333336|
|         51| 33.56603773584906|
|         12| 33.55172413793103|
|         50|              33.5|
|         31| 33.42088091353997|
|         35| 33.41230551626591|
|         17|32.515151515151516|
|         22|32.334916864608076|
|         41| 32.05580357142857|
|         27| 32.03333333333333|
|         52|31.771929824561404|
+-----------+------------------+
only showing top 20 rows



In [137]:
resposta1 = spark.sql('''
    SELECT
    CO_UF_CURSO,
    count(*) 
    FROM temp
    WHERE QE_I03 = 'C'
    group by 1
    order by 2 desc
''')

resposta1.show()

+-----------+--------+
|CO_UF_CURSO|count(1)|
+-----------+--------+
|         35|     360|
|         41|     160|
|         33|     109|
|         31|      78|
|         42|      64|
|         23|      60|
|         43|      52|
|         53|      24|
|         29|      23|
|         50|      22|
|         13|      19|
|         27|      17|
|         52|      17|
|         25|      15|
|         24|      13|
|         26|      12|
|         17|      12|
|         15|      10|
|         22|       9|
|         32|       8|
+-----------+--------+
only showing top 20 rows

